In [5]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [6]:
BATCH_SIZE = 16
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 15

train_ds = image_dataset_from_directory(
    'sdc/train',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_ds = image_dataset_from_directory(
    'sdc/test',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Preprocess using EfficientNet-specific preprocessing
train_ds = train_ds.map(lambda x, y: (tf.keras.applications.efficientnet.preprocess_input(x), y))
val_ds = val_ds.map(lambda x, y: (tf.keras.applications.efficientnet.preprocess_input(x), y))

Found 21000 files belonging to 15 classes.
Found 2715 files belonging to 15 classes.


In [7]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 588s 442ms/step - accuracy: 0.3414 - loss: 2.0731 - val_accuracy: 0.4460 - val_loss: 1.7298 - learning_rate: 0.0010
Epoch 2/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 568s 432ms/step - accuracy: 0.5134 - loss: 1.5242 - val_accuracy: 0.4895 - val_loss: 1.6038 - learning_rate: 0.0010
Epoch 3/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 567s 432ms/step - accuracy: 0.5925 - loss: 1.2738 - val_accuracy: 0.5053 - val_loss: 1.5729 - learning_rate: 0.0010
Epoch 4/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 543s 414ms/step - accuracy: 0.6468 - loss: 1.0878 - val_accuracy: 0.5352 - val_loss: 1.5194 - learning_rate: 0.0010
Epoch 5/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 549s 418ms/step - accuracy: 0.6953 - loss: 0.9341 - val_accuracy: 0.5580 - val_loss: 1.5308 - learning_rate: 0.0010
Epoch 6/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 549s 418ms/step - accuracy: 0.7239 - loss: 0.8332 - val_accuracy: 0.5783 - val_loss: 1.5363 - learning_rate: 0.0010
Epoch 7/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 557s 424ms

In [15]:
train_loss, train_acc = model.evaluate(train_ds)
print(f"Train Accuracy: {train_acc:.4f}")

  34/1313 ━━━━━━━━━━━━━━━━━━━━ 7:42 362ms/step - accuracy: 0.9524 - loss: 0.2153

KeyboardInterrupt: 

In [17]:
test_loss, test_acc = model.evaluate(val_ds)
print(f"Test Accuracy: {test_acc:.4f}")

170/170 ━━━━━━━━━━━━━━━━━━━━ 58s 340ms/step - accuracy: 0.6272 - loss: 1.4616
Test Accuracy: 0.6287
